# Ziad Al-Ziadi

## Testing our CNN model with the EmotionRecognition function

In the preceding section, we define the EmotionRecognition function to use with the CNN we built and trained previously. The CNN will be loaded in as a pickle file.

#NOTE: This notebook was built with GPU and CUDA enabled

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import os
from PIL import Image
import torch.optim as optim
from torch.optim import lr_scheduler
import cv2
from google.colab.patches import cv2_imshow
import pickle

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
####### PLEASE DRAG THE "CV_Submission" FROM YOUR "Shared with me" INTO YOUR "My Drive" OTHERWISE THIS WASN'T WORK #######

import os

MY_OWN_PATH = "/content/drive/MyDrive/CV_Submission/"
MY_OWN_PATH = "CV_Submission"
FULL_PATH = os.path.join("drive", "My Drive", MY_OWN_PATH)
print(os.listdir(FULL_PATH))

In [ ]:
# Defining our function which takes a model and test dataframe

def EmotionRecognition(model, video_input):

  vid = cv2.VideoCapture(video_input)
  
  transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

  if (vid.isOpened() == False):
    print("An error occured while trying to read video")
  frame_width = int(vid.get(3))
  frame_height = int(vid.get(4))


  while(vid.isOpened()):
    ret, frame = vid.read()
    if ret == True:
      model.eval()
      with torch.no_grad():
        
        haar_cas = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces_detected = haar_cas.detectMultiScale(frame, scaleFactor = 1.2, minNeighbors=5)
        
        for (x, y, w, h) in faces_detected:
          cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0),2)

          converted_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
          trans_frame = transform(converted_image)
          trans_frame.cuda()
          model.cuda()

        outputs = model(trans_frame.unsqueeze(0).cuda())
        _, preds = torch.max(outputs.data, 1)

      emo = {0: "Surprise", 1: "Fear", 2: "Disgust", 3: "Happiness", 4: "Sadness", 5: "Anger", 6: "Neutral"}
      cv2.putText(frame, emo[preds.item()], (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0),2)
      cv2_imshow(frame)
      # out.write(frame)
      if cv2.waitKey(1) & 0xFF == ord("q"):
        break
    else:
      break

  vid.release()
  cv2.destroyAllWindows()

In [ ]:
# Download the VGG model class

models.vgg16(pretrained = True)

In [ ]:
# Loading in our CNN

with open("/content/drive/MyDrive/CV_Submission/Test Functions/vgg_cnn.pkl", "rb") as vgg:
    vgg_model = pickle.load(vgg)

In [ ]:
# Here, we have MP4 videos that contain a video for each emotion

happy_vid = "/content/drive/MyDrive/CV_Submission/Videos/happy.mp4"
angry_vid = "/content/drive/MyDrive/CV_Submission/Videos/angry.mp4"
neutral_vid = "/content/drive/MyDrive/CV_Submission/Videos/neutral.mp4"
sad_vid = "/content/drive/MyDrive/CV_Submission/Videos/sad.mp4"
surprised_vid = "/content/drive/MyDrive/CV_Submission/Videos/suprised.mp4"

bond = "/content/drive/MyDrive/CV_Submission/Videos/cr.mp4"
obama = "/content/drive/MyDrive/CV_Submission/Videos/obama.mp4"
interstellar = "/content/drive/MyDrive/CV_Submission/Videos/interstel.mp4"
trump = "/content/drive/MyDrive/CV_Submission/Videos/trump2.mp4"
elon = "/content/drive/MyDrive/CV_Submission/Videos/elon.mp4"

In [ ]:
# Testing our model

EmotionRecognition(vgg_model, bond)